# 正则表达式和 Tkinter 作业

**截止时间：2020 年 11 月 10 日中午 12:00**

---

1. 本次作业题目较为复杂，请务必仔细阅读要求

2. 提交文件命名为 "姓名-学号.ipynb/.txt/.json"，打包的压缩文件也命名为 "姓名-学号.zip"。代码中应当包含必要注释，使其具有较高可读性。


## Python 文档层次化信息提取

1.  使用正则表达式针对 Python 3.9.0 中英文文档中 **faq 目录** 作层次化编码。例如，"Python 常见问题" 编码为 001，"一般信息" 编码为 001，"什么是 Python？" 编码为 001，则完整的层次化问题标签为

    **{Q001001001Zh:"Python 常见问题==>一般信息==>什么是 Python？"}**

    其中 Q 表示问题，Zh 表示中文文档。英文文档编码要求末尾用 En 表示，其他和中文文档相同，例如该问题的英文编码为

    **{Q001001001En:"General Python FAQ==>General Information==>What is Python?"}**

    每个问题下的正文内容作为该问题的回答，和问题采用相同的编码，开头使用字母 A 表示。例如：

    **{A001001001Zh:"Python 是一种解释型、交互式、面向对象的编程语言……"}**

    英文回答同理：

    **{A001001001En:"Python is an interpreted, interactive, object-oriented programming language……"}**

    要求提交编码结果的文本文件，txt/csv 等格式均可。

2.  使用 json 格式表示每个对应层级的编码。通过对应编码，可以将一个问题路径对应到问题编码，也可以由问题编码构造问题路径。参考格式如下：

    ```python
    {
        "title": "faq",  # faq 文件夹
        "zh-title": "Python 常见问题",  # 中文标签
        "level": 1,  # 层级，假设根目录是 0 级，那么 faq 是第 1 级文件夹
        "content": [
            {
                "code": "001",
                "title": "General Python FAQ",  # general.txt 文件
                "zh-title": "Python常见问题",
                "level": 2,  # general.txt 是第 2 级文件，以下层级类推
                "content": [
                    {
                        "code": "001",
                        "title": "General Information",
                        "zh-title": "一般信息",
                        "level": 3,
                        "content": [
                            {
                                "code": "001",
                                "title": "What is Python?",
                                "zh-title": "什么是 Python？",
                                "level": 4,
                                "content": [],
                            },...
                        ],
                    },...
                ],
            },...
        ],
    }
    ```

    要求提交层级编码的 json 文件。

3.  注意：

    - 目录深度不同，编码长度可能不等长；
    - 必须保证每个问题编码唯一；
    - 中文文档目录为中文，文件名为英文，对应关系不明确，建议先参考英文文档建立文件和目录对应关系；
    - 附 1 为 Python 文档简介，附 2 列举了 Python 文档中重要的模式，供参考。


In [1]:
import re,json,os
#dfs进行层次化编码输出到file,并获得每一个标题对应的字典,最后返回
#dep:深度 pos:当前标题在enAll中的位置 enAll,zhAll:英文,中文标题列表 enText,zhText:英文,中文文本
#num:当前标题的编码 strNum:根到当前标题的编码串 lsEnTitle,lsZhTitle:根到当前标题所有标题名称的列表 file:层次化编码输出的文件
def dfs(dep,pos,enAll,zhAll,enText,zhText,num,strNum,lsEnTitle,lsZhTitle,file):
    d={}
    if dep==0:#当前位于faq目录
        d['title']='faq'
        d['zh-title']='Python 常见问题'
    else:
        d['code']='{:0>3d}'.format(num)
        d['title']=enAll[pos][1]
        d['zh-title']=zhAll[pos][1]
        #获取相邻两个标题间的字符串
        enLPos=enAll[pos][0][1]+1
        enRPos=enAll[pos+1][0][0] if pos+1<len(enAll) else len(enText)
        zhLPos=zhAll[pos][0][1]+1
        zhRPos=zhAll[pos+1][0][0] if pos+1<len(zhAll) else len(zhText)
        enAns=enText[enLPos:enRPos].strip(' \n')
        zhAns=zhText[zhLPos:zhRPos].strip(' \n')
        #如果字符串裁去空格回车之后不是空串,则说明这个问题有答案,进行层次化编码并输出
        if enAns!='':
            file.write('{Q'+strNum+'En:'+'"'+'==>'.join(lsEnTitle)+'"}\n')
            file.write('{A'+strNum+'En:'+'"'+enAns+'"}\n')
        if zhAns!='':
            file.write('{Q'+strNum+'Zh:'+'"'+'==>'.join(lsZhTitle)+'"}\n')
            file.write('{A'+strNum+'Zh:'+'"'+zhAns+'"}\n')
    d['level']=dep+1
    d['content']=[]#content初始化为空列表
    tot=0#当前使用的编码
    for i in range(pos+1,len(enAll)):
        if enAll[i][2]==dep+1:#新的标题是当前标题的直接子标题
            tot+=1#编码+1
            d['content'].append(dfs(dep+1,i,enAll,zhAll,enText,zhText,tot,strNum+'{:0>3d}'.format(tot),lsEnTitle+[enAll[i][1]],lsZhTitle+[zhAll[i][1]],file))#对该子标题进行dfs,并将结果追加到content中
        elif enAll[i][2]<=dep:#遇到同级或更高级的标题
            return d
    return d
def solve(enText,zhText):
    #使用re找到所有标题一,二,三级标题
    en1=[(x.span(),x.group()[:x.group().find('\n')],1) for x in re.finditer(r'^.+\n.\*+$',enText,flags=re.MULTILINE)]
    en2=[(x.span(),x.group()[:x.group().find('\n')],2) for x in re.finditer(r'^.+\n.=+$',enText,flags=re.MULTILINE)]
    en3=[(x.span(),x.group()[:x.group().find('\n')],3) for x in re.finditer(r'^.+\n.-+$',enText,flags=re.MULTILINE)]
    zh1=[(x.span(),x.group()[:x.group().find('\n')],1) for x in re.finditer(r'^.+\n.\*+$',zhText,flags=re.MULTILINE)]
    zh2=[(x.span(),x.group()[:x.group().find('\n')],2) for x in re.finditer(r'^.+\n.=+$',zhText,flags=re.MULTILINE)]
    zh3=[(x.span(),x.group()[:x.group().find('\n')],3) for x in re.finditer(r'^.+\n.-+$',zhText,flags=re.MULTILINE)]
    #按出现位置排序
    enAll=sorted(en1+en2+en3,key=lambda x:x[0][0])
    zhAll=sorted(zh1+zh2+zh3,key=lambda x:x[0][0])
    txtFile=open('output.txt','w',encoding='utf8')
    jsonFile=open('output.json','w',encoding='utf8')
    #使用dfs获得字典并转换为json字符串
    strJson=json.dumps(dfs(0,-1,enAll,zhAll,enText,zhText,1,'',[],[],txtFile),indent='\t',ensure_ascii=False)
    jsonFile.write(strJson)
def main1():
    enPath='python-3.9.0-docs-text-en/faq/'
    zhPath='python-3.9.0-docs-text-zh/faq/'
    files=['general.txt','programming.txt','design.txt','library.txt','extending.txt','windows.txt','gui.txt','installed.txt']
    enText,zhText='',''
    #读入所有字符串
    for name in files:
        enText+=open(enPath+name,'r',encoding='utf8').read()
        zhText+=open(zhPath+name,'r',encoding='utf8').read()
    solve(enText,zhText)
main1()

## Tkinter 界面设计

使用 Tkinter 实现检索界面。要求至少包含包含输入框、搜索按钮和结果显示框三个控件。

输入内容可以包含关键字和通配符"?""*"，要求返回检索到的问题标题（不要求显示答案）。

例如，输入"什么是 Python"，要求返回"Python 常见问题==>一般信息==>什么是 Python？"


In [2]:
from tkinter import *
#利用json文件的字典构造所有问题
def builds(d,lsEnTitle,lsZhTitle):
    ans=''
    if d['content']==[]:#没有子标题,说明这是问题
        ans+='==>'.join(lsEnTitle)+'\n'+'==>'.join(lsZhTitle)+'\n'
    for sonD in d['content']:#枚举子标题
        ans+=builds(sonD,lsEnTitle+[sonD['title']],lsZhTitle+[sonD['zh-title']])
    return ans
#查找问题
def search(tk,key,s,text):
    text.delete('1.0','end')#清空结果显示框
    trans=['.','+','$','^','[',']','(',')','{','}','|','\\']#需要转义的字符('*','?'除外)
    newKey=''#转义后的字符串
    for le in key:
        if le in trans:
            newKey+='\\'#添加'\\'
        newKey+=le
    pattern='^.*'+newKey.replace('?','.').replace('*','.*')+'.*$'#构建正则表达式
    ans='\n'.join(re.findall(pattern,s,flags=re.MULTILINE|re.IGNORECASE))#忽略大小写,多行模式匹配
    #显示搜索结果
    if ans=='':
        text.insert(END,'Questions not found!')
    else:
        text.insert(END,ans)
def main2():
    d=json.loads(open('output.json','r',encoding='utf8').read())#从json文件读入数据
    s=builds(d,[],[])#构建问题库
    tk=Tk()
    tk.geometry('800x600+400+300')#设置窗口大小位置
    Label(tk,text='Keywords:').pack(side=TOP,anchor=N,fill=X)#放置一个Label
    et=Entry(tk)#输入框
    et.pack(side=TOP,anchor=N,fill=X)
    text=Text(tk)#结果显示框
    Button(tk,text='Search!',command=lambda:search(tk,et.get(),s,text)).pack(side=TOP,anchor=N,fill=X)#按钮
    text.pack(fill=BOTH,expand=YES)#放置结果显示框
    mainloop()#显示窗口
main2()

## 附录

### 附 1：数据集简介

- Python 3.9.0 中英文文档：

  文件为 plain text，包含多个文件夹和文档。其中 contents.txt 包含全部文件的目录；每个文件夹下的 index.txt 文件包含该文件夹目录。本次作业只用到 faq 文件夹。

  由于纯文本不包含超链接，请务必仔细阅读并充分利用**目录**文件。

### 附 2：信息结构

- 一级标题（文档标题）。每个文件都包含唯一的一级标题，位于文件开头。例如：

  【英文文档】

  ```
  Python Documentation contents
  *****************************
  ```

  【中文文档】

  ```
  Python 文档目录
  ***************
  ```

- 二级标题。每个一级标题后可能有若干个二级标题。例如：

  【英文文档】

  ```
  Contributors to the Python Documentation
  ========================================
  ```

  【中文文档】

  ```
  Python 文档的贡献者
  ===================
  ```

- 三级标题。每个二级标题后可能包含若干个三级标题。例如：

  【英文文档】

  ```
  Assignment expressions
  ----------------------
  ```

  【中文文档】

  ```
  赋值表达式
  ----------
  ```

- 有序列表。例如：

  【英文文档】

  ```
  [1] Since "**" has higher precedence than "-", "-3**2" will be
      interpreted as "-(3**2)" and thus result in "-9".  To avoid this
      and get "9", you can use "(-3)**2".

  [2] Unlike other languages, special characters such as "\n" have the
      same meaning with both single ("'...'") and double (""..."")
      quotes. The only difference between the two is that within single
      quotes you don't need to escape """ (but you have to escape "\'")
      and vice versa.
  ```

  【中文文档】

  ```
  [1] 因为 "**" 比 "-" 有更高的优先级, 所以 "-3**2" 会被解释成 "-(3**2)"
      ，因此结果是 "-9".  为了避免这个并且得到结果 "9", 你可以用这个式子
      "(-3)**2".

  [2] 和其他语言不一样的是, 特殊字符比如说 "\n" 在单引号 ("'...'") 和双
      引号 (""..."") 里有一样的意义. 这两种引号唯一的区别是，你不需要在
      单引号里转义双引号 """ (但是你必须把单引号转义成 "\'") ， 反之亦然
      .
  ```

- 无序列表。例如：

  【英文文档】

  ```
  * What's New in Python
    * What's New In Python 3.9
      * Summary -- Release highlights
      * New Features
        * Assignment expressions
  ```

  【中文文档】

  ```
  * Python的新变化
    * Python 3.9 有什么新变化
      * 摘要 - 发布重点
      * 新的特性
        * 赋值表达式
  ```

- 交互式终端中的 Python 代码块。例如：

  【英文/中文文档】

  ```
  >>> 2 + 2
  4
  >>> 50 - 5*6
  20
  >>> (50 - 5*6) / 4
  5.0
  >>> 8 / 5 # division always returns a floating point number
  1.6
  ```

- 非交互式终端中的 Python 代码块。例如：

  【英文/中文文档】

  ```
  # this is the first comment
  spam = 1 # and this is the second comment
  # ... and now a third!
  text = "# This is not a comment because it's inside quotes."
  ```

- 强调。例如：

  【英文/中文文档】

  ```
  *floor division*
  ```

- 引用。例如：

  【英文/中文文档】

  ```
  "+" "-" "*" "/"
  ```

- 脚注。例如：

  【英文文档】

  ```
  -[ Footnotes ]-
  ```

  【中文文档】

  ```
  -[ 脚注 ]-
  ```
